Import modules for use in this notebook

In [68]:
import requests
import pandas as pd
import json
import os

FOURSQUARE_KEY = os.getenv('FOURSQUARE_API_KEY')


# Foursquare

Send requests to Foursquare with a small radius (1000m) for all the bike stations in Vancouver. 

In [69]:
#Load Vancouver bike station DF from csv
station_df = pd.read_csv('../data/station_df.csv')
station_df = station_df.rename(columns={'slots':'bike_slots'})
station_foursquare_join_df = station_df.copy()
station_foursquare_join_df

,id,name,latitude,longitude,bike_slots
0,00fa94ad698dc4a9e4d708d6fd32f294,Chilco & Barclay,49.291909,-123.140713,18
1,012d3e06901cc222b1c2cf0a2ace3a29,St George & Broadway,49.262321,-123.093060,14
2,029a505bd4422a1afd127987757f71a6,Britannia Parking Lot,49.275882,-123.071865,14
3,0438114d2e6b96118de69bc9775bb21e,Morton & Denman,49.288030,-123.142135,26
4,0459b7e93703980b853cd65a9dc60596,Thornton & National,49.273777,-123.092723,14
...,...,...,...,...,...
253,fbb4c06b719596c21f91fd51ef6d4710,7th & Heather,49.265089,-123.119425,18
254,fe0c57d04d6e682d284d501446095fc9,Union & Dunlevy,49.277595,-123.095830,16
255,fedff7a263c182df94bda7307059cc52,Richards & Helmcken,49.277141,-123.122589,18
256,fef69fb400210d861107a61db954d037,Keefer & Abbott,49.279821,-123.108020,28


In [70]:
#Define FourSquare API function that counts places for each category argument passed

def contact_four_square(row, category, radius=500, limit=50):
    #Base FourSquare endpoint
    foursquare_endpoint = "https://api.foursquare.com/v3/places/search"

    #Authorization headers
    headers = {
        "Accept": "application/json",
        "Authorization": FOURSQUARE_KEY
    }

    #Include search paramters
    #Latitude and Longitude string for params    
    params = {
        'limit': limit,
        'radius': radius,
        'll': f"{row['latitude']},{row['longitude']}",
        'categories': category,
        'fields': 'fsq_id,name,rating,popularity,price,venue_reality_bucket',
    }

    #Contact API
    foursquare_response = requests.get(foursquare_endpoint, headers=headers, params=params)

    #Check url for endpoint with params
    # print(foursquare_response.url)

    foursquare_list = foursquare_response.json()['results']
    place_count = len(foursquare_list)

    return place_count



In [71]:
#Interesting category numbers
categories = {
    'public_art': 10047,
    'night_club': 10032,
    'bicycle_store': 17119,
    'coffee_shop': 13032,
    'fast_food': 13145,
    'museum': 10027,
    'grocery_store': 17069,
}

# Comment out API call to avoid accidental run

#Iterate through all category names, using the id to make API calls to FourSquare and apply to station dataframe
for cat_name, id in categories.items():
    station_foursquare_join_df[cat_name] = station_foursquare_join_df.apply(contact_four_square, axis=1, args=(id,))



Write full data frame to CSV to preserve data.

In [74]:
#View station data frame with point of interest counts
#NOTE: categories have 50 MAX items due to limitations in API call
station_foursquare_join_df

,id,name,latitude,longitude,bike_slots,public_art,night_club,bicycle_store,coffee_shop,fast_food,museum,grocery_store
0,00fa94ad698dc4a9e4d708d6fd32f294,Chilco & Barclay,49.291909,-123.140713,18,0,1,5,16,14,1,11
1,012d3e06901cc222b1c2cf0a2ace3a29,St George & Broadway,49.262321,-123.093060,14,1,2,1,10,3,0,8
2,029a505bd4422a1afd127987757f71a6,Britannia Parking Lot,49.275882,-123.071865,14,0,1,4,21,1,0,12
3,0438114d2e6b96118de69bc9775bb21e,Morton & Denman,49.288030,-123.142135,26,1,1,1,16,11,0,8
4,0459b7e93703980b853cd65a9dc60596,Thornton & National,49.273777,-123.092723,14,0,0,0,7,4,0,11
...,...,...,...,...,...,...,...,...,...,...,...,...
253,fbb4c06b719596c21f91fd51ef6d4710,7th & Heather,49.265089,-123.119425,18,1,0,2,29,9,0,5
254,fe0c57d04d6e682d284d501446095fc9,Union & Dunlevy,49.277595,-123.095830,16,0,1,2,21,8,2,30
255,fedff7a263c182df94bda7307059cc52,Richards & Helmcken,49.277141,-123.122589,18,1,24,4,50,32,2,29
256,fef69fb400210d861107a61db954d037,Keefer & Abbott,49.279821,-123.108020,28,4,15,2,50,16,11,16


In [75]:
#Write data frame to csv to preserve contents
station_foursquare_join_df.to_csv('../data/foursquare_df.csv', index=False)

# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [60]:
import os
import requests
import pandas as pd
import json


YELP_API_KEY = os.getenv('YELP_API_KEY')

In [18]:
#Load City Bikes CSV into new dataframe 
station_df_yelp = pd.read_csv('../data/station_df.csv')

station_df_yelp

,id,name,latitude,longitude,slots
0,00fa94ad698dc4a9e4d708d6fd32f294,Chilco & Barclay,49.291909,-123.140713,18
1,012d3e06901cc222b1c2cf0a2ace3a29,St George & Broadway,49.262321,-123.093060,14
2,029a505bd4422a1afd127987757f71a6,Britannia Parking Lot,49.275882,-123.071865,14
3,0438114d2e6b96118de69bc9775bb21e,Morton & Denman,49.288030,-123.142135,26
4,0459b7e93703980b853cd65a9dc60596,Thornton & National,49.273777,-123.092723,14
...,...,...,...,...,...
253,fbb4c06b719596c21f91fd51ef6d4710,7th & Heather,49.265089,-123.119425,18
254,fe0c57d04d6e682d284d501446095fc9,Union & Dunlevy,49.277595,-123.095830,16
255,fedff7a263c182df94bda7307059cc52,Richards & Helmcken,49.277141,-123.122589,18
256,fef69fb400210d861107a61db954d037,Keefer & Abbott,49.279821,-123.108020,28


Connect to Yelp API 

In [52]:
#Create function to contact Yelp API
def contact_yelp_api(row, category, radius=1000, limit=50):
    #Establish variables for API call
    yelp_endpoint = 'https://api.yelp.com/v3/businesses/search'
    headers = {
        "accept": "application/json",
        "authorization": f"Bearer {YELP_API_KEY}"
    }
    params = {
        'latitude': row['latitude'],
        'longitude': row['longitude'],
        'radius': radius, 
        'limit': limit,
        'term': category,
        'sort_by': 'rating',
    }

    #Request data from API
    yelp_response = requests.get(yelp_endpoint, headers=headers, params=params)

    #Create JSON from response
    yelp_json = yelp_response.json()

    #Return number of businesses returned 
    return len(yelp_json['businesses'])



In [58]:
#Apply number of restaurants & grocery stores to each row in City Bikes DF
station_df_yelp['restaurants'] = station_df_yelp.apply(contact_yelp_api, axis=1, args=('restaurant',))
station_df_yelp['grocery'] = station_df_yelp.apply(contact_yelp_api, axis=1, args=('grocery',))
station_df_yelp

,id,name,latitude,longitude,slots,restaurants,grocery
0,00fa94ad698dc4a9e4d708d6fd32f294,Chilco & Barclay,49.291909,-123.140713,18,50,50
1,012d3e06901cc222b1c2cf0a2ace3a29,St George & Broadway,49.262321,-123.093060,14,50,50
2,029a505bd4422a1afd127987757f71a6,Britannia Parking Lot,49.275882,-123.071865,14,50,50
3,0438114d2e6b96118de69bc9775bb21e,Morton & Denman,49.288030,-123.142135,26,50,50
4,0459b7e93703980b853cd65a9dc60596,Thornton & National,49.273777,-123.092723,14,50,50
...,...,...,...,...,...,...,...
253,fbb4c06b719596c21f91fd51ef6d4710,7th & Heather,49.265089,-123.119425,18,50,50
254,fe0c57d04d6e682d284d501446095fc9,Union & Dunlevy,49.277595,-123.095830,16,50,50
255,fedff7a263c182df94bda7307059cc52,Richards & Helmcken,49.277141,-123.122589,18,50,50
256,fef69fb400210d861107a61db954d037,Keefer & Abbott,49.279821,-123.108020,28,50,50


Export new data frame to CSV to preserve data

In [59]:
station_df_yelp.to_csv('../data/yelp_df.csv', index=False)

Put your parsed results into a DataFrame

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Yelp has more detailed response in their API with the variety of fields used, however the 'categories' feature is hidden in the "developer beta" section which may not be stable for use. Both APIs have a limit of 50 results per search. FourSquare was simpler to use to define desired categories as their list was laid out in an easily parseable manner. 

The FourSquare API allows the user to select which fields are returned, making the results JSON far easier to parse as it can be customized and simplified. 

The Yelp API allows for incorrect arguments to be used and still return a result. While this seems useful, it is preferable to get an error with incorrect arguments to know when a mistake has been made. 

The Yelp API allows for sorting of results which can help get more valid results.

For the purpose of this project, I am concerned with raw numbers of places of interest within my desired catgories so I will use the FourSquare API. For future uses with a much more detailed dataset, the Yelp API may be preferable. 

Get the top 10 restaurants according to their rating

In [ ]:
#Establish variables for API call
yelp_endpoint = 'https://api.yelp.com/v3/businesses/search'
headers = {
    "accept": "application/json",
    "authorization": f"Bearer {YELP_API_KEY}"
}
params = {
    'location': 'Vancouver',
    'limit': 10,
    'term': 'restaurant',
    'sort_by': 'rating',
}

#Call API
yelp_response = requests.get(yelp_endpoint, headers=headers, params=params)

#Create array of businesses
restaurant_list = yelp_response.json()['businesses']

#Print name of each restaurant in list with rating
for restaurant in restaurant_list:
    print(restaurant['name'], restaurant['rating'])

mizu sushi co. 5.0
Nell's Kitchen 5.0
Kim's Cafe 5.0
Sushi Master 5.0
Burdy 5.0
Thank You Pizza 5.0
Santo Taco 5.0
Wren Cafe 5.0
Cockney Kings Fish & Chips 4.9
Lucifer's House of Heat 4.9
